# FBS unexpected behavior on 2024-11-24

The expected behavior: 

using the [fbs_config_minimal_fieldsurvey.py](https://github.com/lsst-ts/ts_config_ocs/blob/7a60941ef82c949a9b0be724e4e7198553cd8065/Scheduler/feature_scheduler/maintel/fbs_config_minimal_fieldsurvey.py) configuration, 
the observers would slew to the field to be observed first tonight - 47_Tuc. The FBS configuration is primarily slew-driven, so then 47_Tuc would be the chosen field.
This was the case (after a small adjustment in pointing location to the correct location). 
However, the configuration was to take 5 images in r band, then 5 images in i band. 

Instead, the acquired observations were 15 images in r band, then 6 in i band.  -- missing i band visits for the first two sets of r band visits.

In [1]:
import os
import numpy as np
import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.summit.utils import ConsDbClient

import sys
sys.path.insert(0, '/home/l/lynnej/u/repos/rubin_scheduler')

import pickle
from urllib.parse import urlparse
from lsst.resources import ResourcePath
from lsst_efd_client import EfdClient
from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site
from rubin_scheduler.scheduler.example import get_ideal_model_observatory

from rubin_scheduler.scheduler.schedulers import CoreScheduler
from rubin_scheduler.scheduler.features import Conditions

os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/shared/rubin_sim_data'

days_to_seconds = 24*60*60

In [2]:
# Verify visits acquired. 

os.environ["LSST_CONSDB_PQ_URL"] = "http://consdb-pq.consdb:8080/consdb"
os.environ["no_proxy"] += ",.consdb"

day_obs = "2024-11-24"

consdb = ConsDbClient()
instrument = 'lsstcomcam'
day_obs_int = day_obs.replace('-', '')
visit_query = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''
visits = consdb.query(visit_query).to_pandas()

vv = pd.DataFrame(visits.query('science_program == "BLOCK-320"'))
delta_visit = vv.exp_midpt_mjd.values[1:] - vv.exp_midpt_mjd.values[:-1]
delta_visit = np.concatenate([np.array([0]), delta_visit])
vv['delta_visit'] = delta_visit * 24 * 60 * 60
print(vv.exp_midpt_mjd.min(), vv.exp_midpt_mjd.max(), (vv.exp_midpt_mjd.max() - vv.exp_midpt_mjd.min()) * 24 * 60)
vcols = ['exposure_name', 'group_id', 'delta_visit', 'exp_midpt', 'exp_midpt_mjd', 'band', 's_ra', 's_dec', 'sky_rotation', 'altitude', 'azimuth', 'target_name',]
display(vv[vcols])
display(vv[['exp_midpt', 'band', 'target_name']].groupby(['target_name', 'band']).count())

60639.03863641126 60639.274200117856 339.2117375030648


,exposure_name,group_id,delta_visit,exp_midpt,exp_midpt_mjd,band,s_ra,s_dec,sky_rotation,altitude,azimuth,target_name
93,CC_O_20241124_000094,2024-11-25T00:54:43.804,0.000000,2024-11-25T00:55:38.186000,60639.038636,r,5.958926,-49.251333,21.252060,18.894173,-177.637683,Abell_2764
100,CC_O_20241124_000101,2024-11-25T01:07:31.940,748.741643,2024-11-25T01:08:06.927500,60639.047302,r,6.432868,-71.992058,7.307100,41.667498,-178.185460,47_Tuc
101,CC_O_20241124_000102,2024-11-25T01:07:32.929,49.493380,2024-11-25T01:08:56.420500,60639.047875,r,6.544144,-72.083343,7.154104,41.761093,-178.152862,47_Tuc
102,CC_O_20241124_000103,2024-11-25T01:08:33.484,50.602345,2024-11-25T01:09:47.023500,60639.048461,r,6.559047,-72.075984,7.134961,41.759383,-178.060872,47_Tuc
103,CC_O_20241124_000104,2024-11-25T01:09:22.935,60.739447,2024-11-25T01:10:47.762500,60639.049164,r,6.100476,-72.118005,7.724047,41.823572,-177.739459,47_Tuc
104,CC_O_20241124_000105,2024-11-25T01:10:14.564,51.818806,2024-11-25T01:11:39.581500,60639.049764,r,5.813753,-72.105914,8.095254,41.829487,-177.506428,47_Tuc
105,CC_O_20241124_000106,2024-11-25T01:11:15.192,53.199099,2024-11-25T01:12:32.780500,60639.050379,r,6.342651,-72.018952,8.345849,41.732121,-177.631103,47_Tuc
106,CC_O_20241124_000107,2024-11-25T01:12:06.413,65.733182,2024-11-25T01:13:38.513500,60639.051140,r,6.333955,-72.181985,8.336802,41.904343,-177.529273,47_Tuc
107,CC_O_20241124_000108,2024-11-25T01:12:59.652,51.797252,2024-11-25T01:14:30.311500,60639.051740,r,6.535170,-72.070577,8.090860,41.794422,-177.501447,47_Tuc
108,CC_O_20241124_000109,2024-11-25T01:14:04.899,51.548540,2024-11-25T01:15:21.859500,60639.052336,r,6.534662,-71.989363,8.101208,41.722167,-177.386413,47_Tuc


exp_midpt
target_name band           
47_Tuc      i             6
            r            15
Abell_2764  r             1
Fornax_dSph i             5
            r            21

In [3]:
# Set a range of times based on these visits values to query efd and consdb .. 

t_start = Time("2024-11-25T00:54:43.804", format='isot', scale='utc') - TimeDelta(15 * 60 , format='sec')
t_end = Time("2024-11-25T01:25:08.398", format='isot', scale='utc') + TimeDelta(10 * 60, format='sec')

print(f"Querying from {t_start.iso} to {t_end.iso}")

Querying from 2024-11-25 00:39:43.804 to 2024-11-25 01:35:08.398


In [4]:
# Get targets from the EFD 

efd = 'usdf_efd'

efd_client = EfdClient(efd)

salindex = 1
topic = 'lsst.sal.Scheduler.logevent_target'
fields = await efd_client.get_fields(topic)
targets = await efd_client.select_time_series(topic, fields, t_start, t_end, index=salindex)
delta_target = (targets.index.values[1:] - targets.index.values[:-1]) / np.timedelta64(1, 's')
targets['delta_target'] = np.concatenate([np.array([0]), delta_target])
targets['count'] = np.arange(0, len(targets))
#print(fields)
tcols = ['delta_target', 'requestMjd', 'ra', 'decl', 'filter', 'exposureTimes0', 'slewTime', 'skyAngle', 'airmass', 'moonRa', 'skyBrightness', 'seeing', 'note', 'targetName', 'targetId', 'count']
targets[tcols]#.query('count >=3')

# Note that targets 0 - 2 are for the wrong pointing .. the observers had to move to the correct field first. (one of these targets was executed, by mistake). 

,delta_target,requestMjd,ra,decl,filter,exposureTimes0,slewTime,skyAngle,airmass,moonRa,skyBrightness,seeing,note,targetName,targetId,count
2024-11-25 00:53:57.957974+00:00,0.000000,0,5.907077,-49.159884,r_03,30,15.005228,21.409428,1.061105,0,21.276972,0,,,0,0
2024-11-25 00:53:59.253444+00:00,1.295470,0,5.958926,-49.251333,r_03,30,14.954363,21.254447,1.061105,0,21.276972,0,,,0,1
2024-11-25 00:54:21.947333+00:00,22.693889,0,5.749787,-49.285571,r_03,30,14.929433,21.800130,1.061105,0,21.276972,0,,,0,2
2024-11-25 01:06:47.375424+00:00,745.428091,0,6.432868,-71.992058,r_03,30,3.226085,7.308640,1.352043,0,21.140179,0,,,0,3
2024-11-25 01:06:48.400796+00:00,1.025372,0,6.544144,-72.083343,r_03,30,3.389077,7.155248,1.352043,0,21.140179,0,,,0,4
2024-11-25 01:07:49.048891+00:00,60.648095,0,6.559047,-72.075984,r_03,30,3.377532,7.136721,1.352043,0,21.140179,0,,,0,5
2024-11-25 01:08:38.538145+00:00,49.489254,0,6.100476,-72.118005,r_03,30,3.453587,7.724922,1.352043,0,21.140179,0,,,0,6
2024-11-25 01:09:30.100967+00:00,51.562822,0,5.813753,-72.105914,r_03,30,3.449957,8.096665,1.352043,0,21.140179,0,,,0,7
2024-11-25 01:10:30.624009+00:00,60.523042,0,6.342651,-72.018952,r_03,30,3.360069,8.347103,1.352787,0,21.140790,0,,,0,8
2024-11-25 01:11:21.902625+00:00,51.278616,0,6.333955,-72.181985,r_03,30,3.378041,8.338119,1.352787,0,21.140790,0,,,0,9


In [5]:
# Get the scheduler snapshots 

topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
fields = ["url"]

snapshots = await efd_client.select_time_series(topic, fields, t_start, t_end, index=salindex)
print(len(snapshots))
display(HTML(snapshots.to_html()))

4


,url
2024-11-25 00:53:56.106745+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/24/Scheduler:1_Scheduler:1_2024-11-25T00:54:32.921.p
2024-11-25 01:06:45.440706+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/24/Scheduler:1_Scheduler:1_2024-11-25T01:07:21.450.p
2024-11-25 01:14:14.342994+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/24/Scheduler:1_Scheduler:1_2024-11-25T01:14:50.574.p
2024-11-25 01:23:33.037290+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/24/Scheduler:1_Scheduler:1_2024-11-25T01:24:09.664.p


In [6]:
np.diff(snapshots.index.values) / np.timedelta64(1,'s') / 50
# snapshots are about 10 visits apart .. but only the r band visits are being acquired, so it's still two sequences.

array([15.38667922,  8.97804576, 11.17388592])

In [7]:
# we want the SECOND one of these, corresponding to the request at the proper target
index = 1

snapshot_time = snapshots.iloc[index].name
snapshot_mjd = Time(snapshot_time).mjd
url = snapshots.iloc[index].url
print(snapshot_time, url)

# EFD records the summit LFA -- if not at the summit, swap.
bucket = "s3://rubin:"
uri = ResourcePath(bucket + urlparse(url).path.lstrip('/'))

# multi-tenant names have colons 
os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"

result = uri.read()

#unpickle
sched, conditions = pickle.loads(result)

# Just check that these are the right kind of things 
assert isinstance(sched, CoreScheduler)
assert isinstance(conditions, Conditions)

2024-11-25 01:06:45.440706+00:00 https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/24/Scheduler:1_Scheduler:1_2024-11-25T01:07:21.450.p


In [8]:
#sched.survey_lists

In [9]:
# Check conditions time, and visit to the target 
print('conditions time', conditions.mjd) 
# Check on alt/az limits in the conditions (to see if anything masked)
print('tel limits', np.degrees(conditions.tel_alt_limits), np.degrees(conditions.tel_az_limits))
# And check on current position of the telescope (especially due to this FBS configuration)
print('tel pos', np.degrees(conditions.tel_ra), np.degrees(conditions.tel_dec))
# Filters?
print(conditions.current_filter, conditions.mounted_filters)

conditions time 60639.04677083333
tel limits [40. 84.] [-200.  200.]
tel pos 6.477305519157878 -71.94839877993121
r_03 ['i_06', 'r_03', 'y_04']


In [10]:
# just check on surveys and how many observations they know about .. 
# (there's only one tier in this scheduler configuration)
# Might as well check total # of visits against consdb previous visits 
# (since this is start of the day, these should be all from previous nights)
# visit_query = f'''SELECT * FROM cdb_{instrument}.visit1 where day_obs < {day_obs_int} and science_program like '%%BLOCK-320%%' '''
# visits = consdb.query(visit_query).to_pandas()

# for s in sched.survey_lists[0]:
#     # ok, the list is long .. only do this for the ones with non-zero obs for the survey .. 
#     if s.extra_features['ObsRecorded_note'].feature > -1:
#         print("survey", s.survey_name, np.degrees(s.ra), np.degrees(s.dec))
#         # hmm .. not all previous visits are counted .. but perhaps some of the previous visits were bad?
#         print("nvisits", s.extra_features['ObsRecorded'].feature, s.extra_features['ObsRecorded_note'].feature, len(visits.query('target_name == @s.survey_name')))

In [11]:
# Check times, looking at lag between visits and target and conditions
visitidx = 100
targetidx = 3
print(conditions.mjd, vv.exp_midpt_mjd[visitidx], Time(targets.index[targetidx]).utc.mjd)
print("first visit - conditions time")
print((vv.exp_midpt_mjd[visitidx] - conditions.mjd) * days_to_seconds )
print("conditions - target recorded time")
print((conditions.mjd - Time(targets.index[targetidx]).utc.mjd) * days_to_seconds)

60639.04677083333 60639.0473024025 60639.04638166
first visit - conditions time
45.92757602222264
conditions - target recorded time
33.62457558978349


In [12]:
# All right - let's go update the scheduler to the conditions, and ask for an observation. 
print(sched.conditions.mjd, conditions.mjd, sched.conditions.tel_ra)
sched.update_conditions(conditions)
print(sched.conditions.mjd, conditions.mjd, sched.conditions.tel_ra)
# ... so maybe at the start of the night, the conditions pickled in the scheduler match the current conditions. 

# Also, given the times above .. the conditions.mjd is not quite the time the visit is acquired, 
# but it's also not the time the target is recorded .. so let's assume it's the goal visit start time.

60639.04677083333 60639.04677083333 0.11305030796690561
60639.04677083333 60639.04677083333 0.11305030796690561


In [13]:
# Find observations that the scheduler would request
obs = sched.request_observation(conditions.mjd, whole_queue=True)

/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:292: RuntimeWarning: All-NaN axis encountered
  rewards[i] = np.nanmax(survey.calc_reward_function(self.conditions))


In [14]:
print(len(obs))
for o in obs:
    print(o['filter'], np.degrees(o['rotTelPos']), np.degrees(o['rotSkyPos']))

10
['r_03'] [-1.94549728] [7.30322847]
['r_03'] [-1.94549728] [7.14984607]
['r_03'] [-1.94549728] [7.13131759]
['r_03'] [-1.94549728] [7.71952519]
['r_03'] [-1.94549728] [8.09126834]
['i_06'] [-22.13662077] [27.66799006]
['i_06'] [-22.13662077] [28.20918298]
['i_06'] [-22.13662077] [28.52429797]
['i_06'] [-22.13662077] [27.7188116]
['i_06'] [-22.13662077] [28.14472807]


In [15]:
# I considered that the rotator angle might be out of range, but the __init__.yaml and the __summit.yaml files in ts_config_ocs suggest that the allowed rotator range should be -60 to 60.
# The angles here match the angles from rubintv for the first visit, so the FBS requests seem to be in range. 
# Also, the final i band visits actually have a similar sort of angle as to the requested values here for i band.

In [16]:
# So the FBS asked for the correct observations .. what happened to the i band visits? 
# Let's check the scheduler logs:

# the logs are plentiful, so let's zoom in to the time of the first good sequence - the time around target #3
tstart = Time('2024-11-25 01:06:47.375424', format='iso', scale='utc') - TimeDelta(10, format='sec')
tend = Time('2024-11-25 01:13:20.388519', format='iso', scale='utc') 

topic = 'lsst.sal.Scheduler.logevent_logMessage'
fields = ['functionName', 'level', 'lineNumber' ,'message', 'salIndex']
log = await efd_client.select_time_series(topic, fields, tstart, tend, index=1)
display(HTML(log.to_html()))

# There are a lot of the scheduler rejecting the i-band targets from the first two sequences, with error 64 .. 
# Tracking this back, it's error  64 from ts_observatory_model, indicating that the filter change is not possible. 
# The reason for this isn't obvious .. filter burst seems most likely, but the filter_burst_time seems like it's set to 0, so should be fine.
# the average filter numbers are also averaged over a year, with a limit of 30,000 .. which also seems unlikely to have been hit.
# The name of the filter seems fine too (and executed in the last sequence). 

,functionName,level,lineNumber,message,salIndex
2024-11-25 01:06:43.387879+00:00,do_resume,20,448,Resuming Scheduler operations...,1
2024-11-25 01:06:44.390035+00:00,check_scheduled_targets,20,1360,No scheduled targets to check.,1
2024-11-25 01:06:45.619243+00:00,_get_validated_target_from_observation,40,425,"Error[64]: Cannot slew to target @ ra=6.289501637090249, dec=-72.10384189885859.\ntarget=targetid=0 field=0 filter=i_06 exp_times=[30.0] ra=6.290 dec=-72.104 ang=27.673 alt=0.000 az=0.000 rot=-22.137 telalt=0.000 telaz=0.000 telrot=0.000 time=0.0 airmass=0.000 brightness=0.000 cloud=0.00 seeing=0.00 visits=0 progress=0.00% seqid=0 ssname= groupid=0 groupix=0 firstdd=False ddvisits=0 need=0.000 bonus=0.000 value=0.000 propboost=1.000 propid=[] need=[] bonus=[] value=[] propboost=[] slewtime=0.000 cost=0.000 rank=0.000 note=.\nobservation=array([(0, 0.10977251, -1.25844944, 0., 60642.82761574, 30., 'i_06', 0.48299175, 0., 1, 0., 0., 0., 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., '', '47_Tuc', '47_Tuc', 0, 0., -0.38635692, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 'science', 'BLOCK-320')],\n dtype=[('ID', '<i8'), ('RA', '<f8'), ('dec', '<f8'), ('mjd', '<f8'), ('flush_by_mjd', '<f8'), ('exptime', '<f8'), ('filter', '<U40'), ('rotSkyPos', '<f8'), ('rotSkyPos_desired', '<f8'), ('nexp', '<i8'), ('airmass', '<f8'), ('FWHM_500', '<f8'), ('FWHMeff', '<f8'), ('FWHM_geometric', '<f8'), ('skybrightness', '<f8'), ('night', '<i8'), ('slewtime', '<f8'), ('visittime', '<f8'), ('slewdist', '<f8'), ('fivesigmadepth', '<f8'), ('alt', '<f8'), ('az', '<f8'), ('pa', '<f8'), ('pseudo_pa', '<f8'), ('clouds', '<f8'), ('moonAlt', '<f8'), ('sunAlt', '<f8'), ('note', '<U40'), ('scheduler_note', '<U40'), ('target_name', '<U40'), ('block_id', '<i8'), ('lmst', '<f8'), ('rotTelPos', '<f8'), ('rotTelPos_backup', '<f8'), ('moonAz', '<f8'), ('sunAz', '<f8'), ('sunRA', '<f8'), ('sunDec', '<f8'), ('moonRA', '<f8'), ('moonDec', '<f8'), ('moonDist', '<f8'), ('solarElong', '<f8'), ('moonPhase', '<f8'), ('cummTelAz', '<f8'), ('observation_reason', '<U40'), ('science_program', '<U40')]).\nObservatory State:t=1732496842.5 ra=6.476 dec=-71.948 ang=8.050 filter=r_03 track=True alt=48.238 az=181.904 pa=5.313 rot=357.264 telaz=-178.096 telrot=-2.736 mounted=['i_06', 'r_03', 'y_04'] unmounted=['u_02', 'g_01', 'z_03'].\n",1
2024-11-25 01:06:45.619454+00:00,_get_validated_target_from_observation,40,425,"Error[64]: Cannot slew to target @ ra=5.879193303288013, dec=-72.00985736361403.\ntarget=targetid=0 field=0 filter=i_06 exp_times=[30.0] ra=5.879 dec=-72.010 ang=28.215 alt=0.000 az=0.000 rot=-22.137 telalt=0.000 telaz=0.000 telrot=0.000 time=0.0 airmass=0.000 brightness=0.000 cloud=0.00 seeing=0.00 visits=0 progress=0.00% seqid=0 ssname= groupid=0 groupix=0 firstdd=False ddvisits=0 need=0.000 bonus=0.000 value=0.000 propboost=1.000 propid=[] need=[] bonus=[] value=[] propboost=[] slewtime=0.000 cost=0.000 rank=0.000 note=.\nobservation=array([(0, 0.10261128, -1.2568091, 0., 60642.82761574, 30., 'i_06', 0.49243749, 0., 1, 0., 0., 0., 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., '', '47_Tuc', '47_Tuc', 0, 0., -0.38635692, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 'science', 'BLOCK-320')],\n dtype=[('ID', '<i8'), ('RA', '<f8'), ('dec', '<f8'), ('mjd', '<f8'), ('flush_by_mjd', '<f8'), ('exptime', '<f8'), ('filter', '<U40'), ('rotSkyPos', '<f8'), ('rotSkyPos_desired', '<f8'), ('nexp', '<i8'), ('airmass', '<f8'), ('FWHM_500', '<f8'), ('FWHMeff', '<f8'), ('FWHM_geometric', '<f8'), ('skybrightness', '<f8'), ('night', '<i8'), ('slewtime', '<f8'), ('visittime', '<f8'), ('slewdist', '<f8'), ('fivesigmadepth', '<f8'), ('alt', '<f8'), ('az', '<f8'), ('pa', '<f8'), ('pseudo_pa', '<f8'), ('clouds', '<f8'), ('moonAlt', '<f8'), ('sunAlt', '<f8'), ('note', '<U40'), ('scheduler_note', '<U40'), ('target_name', '<U40'), ('block_id', '<i8'), ('lmst', '<f8'), ('rotTelPos', '<f8'), ('rotTelPos_backup', '<f8'), ('moonAz', '<f8'), ('sunAz', '<f8'), ('sunRA', '<f8'), ('sun

In [17]:
# the error message regarding i band targets ..
print(log.iloc[2].message)

Error[64]: Cannot slew to target @ ra=6.289501637090249, dec=-72.10384189885859.
target=targetid=0 field=0 filter=i_06 exp_times=[30.0] ra=6.290 dec=-72.104 ang=27.673 alt=0.000 az=0.000 rot=-22.137 telalt=0.000 telaz=0.000 telrot=0.000 time=0.0 airmass=0.000 brightness=0.000 cloud=0.00 seeing=0.00 visits=0 progress=0.00% seqid=0 ssname= groupid=0 groupix=0 firstdd=False ddvisits=0 need=0.000 bonus=0.000 value=0.000 propboost=1.000 propid=[] need=[] bonus=[] value=[] propboost=[] slewtime=0.000 cost=0.000 rank=0.000 note=.
observation=array([(0, 0.10977251, -1.25844944, 0., 60642.82761574, 30., 'i_06', 0.48299175, 0., 1, 0., 0., 0., 0., 0., 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., '', '47_Tuc', '47_Tuc', 0, 0., -0.38635692, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 'science', 'BLOCK-320')],
      dtype=[('ID', '<i8'), ('RA', '<f8'), ('dec', '<f8'), ('mjd', '<f8'), ('flush_by_mjd', '<f8'), ('exptime', '<f8'), ('filter', '<U40'), ('rotSkyPos', '<f8'), ('rotSkyPos_desired', '<f8')

This is a message from the ts_scheduler code, coming from [here](https://github.com/lsst-ts/ts_scheduler/blob/develop/python/lsst/ts/scheduler/driver/feature_scheduler.py#L415). 
It indicates that the ts_observatory_model "get_slew_delay" call rejected the observation as impossible. The error code 64 means that it was rejected due to the filter change. 

In this case, Tiago indicated that it was the filter burst rate calculation that was causing it to fail, although I'm not sure why that failed today and not yesterday, nor why it worked for the third sequence. Perhaps small changes in the time and state of the observatory model?
Increasing the `filter_max_changes_burst_num` seemed to solve the problem (https://github.com/lsst-ts/ts_config_ocs/commit/c7912e990ff8a77057cf8bdd20ea152326d09859). 

In [18]:
index = 2  # Corresponds to the *third* sequence, which is where i band was finally allowed

snapshot_time = snapshots.iloc[index].name
snapshot_mjd = Time(snapshot_time).mjd
url = snapshots.iloc[index].url
print(snapshot_time, url)

# EFD records the summit LFA -- if not at the summit, swap.
bucket = "s3://rubin:"
uri = ResourcePath(bucket + urlparse(url).path.lstrip('/'))

# multi-tenant names have colons 
os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"

result = uri.read()

#unpickle
sched, conditions = pickle.loads(result)

# Just check that these are the right kind of things 
assert isinstance(sched, CoreScheduler)
assert isinstance(conditions, Conditions)


2024-11-25 01:14:14.342994+00:00 https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/24/Scheduler:1_Scheduler:1_2024-11-25T01:14:50.574.p


In [19]:
conditions.mounted_filters, conditions.current_filter

(['i_06', 'r_03', 'y_04'], 'r_03')

In [20]:
sched.update_conditions(conditions)
obs2 = sched.request_observation(conditions.mjd, whole_queue=True)

/home/l/lynnej/u/repos/rubin_scheduler/rubin_scheduler/scheduler/schedulers/core_scheduler.py:292: RuntimeWarning: All-NaN axis encountered
  rewards[i] = np.nanmax(survey.calc_reward_function(self.conditions))


In [21]:
# This sequence succeeded in acquiring i band
len(obs2)
for o in obs:
    print(o['filter'], np.degrees(o['rotTelPos']), np.degrees(o['rotSkyPos']))

['r_03'] [-1.94549728] [7.30322847]
['r_03'] [-1.94549728] [7.14984607]
['r_03'] [-1.94549728] [7.13131759]
['r_03'] [-1.94549728] [7.71952519]
['r_03'] [-1.94549728] [8.09126834]
['i_06'] [-22.13662077] [27.66799006]
['i_06'] [-22.13662077] [28.20918298]
['i_06'] [-22.13662077] [28.52429797]
['i_06'] [-22.13662077] [27.7188116]
['i_06'] [-22.13662077] [28.14472807]


In [22]:
# Later visits to another field  showed the same problem, until the max filter burst rate number was increased. 